#### Note: 

 This section was originally written to play around with the data from a single participant. 

In [ ]:
import os, sys

print(sys.executable) # This is where this notebook is running in
print(sys.path) # This is where the downloaded modules are - sys.executable and sys.path need to be the same to be able to load the module.

# Solution:
# Run this in Terminal: (output of sys.executable) -m pip install (package name)

In [ ]:
# Load packages to convert formats
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
import pingouin
from sklearn import preprocessing
import numpy as np
from numpy.core import einsumfunc
import utility as ut

In [ ]:
# File where the NMA fMRI data are
path_data = "/Volumes/DPhil_Jelka/fmri_data/hcp_task"

try: 
    os.chdir(path_data) 
except OSError as error: 
    print(error)  

In [ ]:
# Check if you successfully changed directory
os.getcwd() 

In [ ]:
# Okay, you want hcp_task to be your specific directory for now
HCP_DIR = "/Volumes/DPhil_Jelka/fmri_data/hcp_task"
if not os.path.isdir(HCP_DIR):
    os.mkdir(HCP_DIR)

# The data shared for NMA projects is a subset of the full HCP dataset (0-338)
N_SUBJECTS = 339

# The data have already been aggregated into ROIs from the Glasser parcellation
N_PARCELS = 360

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in seconds

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated twice in each subject
N_RUNS = 2

# There are 7 tasks. Each has a number of 'conditions'

EXPERIMENTS = {
    'MOTOR'      : {'runs': [5,6],   'cond':['lf','rf','lh','rh','t','cue']},
    'WM'         : {'runs': [7,8],   'cond':['0bk_body','0bk_faces','0bk_places','0bk_tools','2bk_body','2bk_faces','2bk_places','2bk_tools']},
    'EMOTION'    : {'runs': [9,10],  'cond':['fear','neut']},
    'GAMBLING'   : {'runs': [11,12], 'cond':['loss','win']},
    'LANGUAGE'   : {'runs': [13,14], 'cond':['math','story']},
    'RELATIONAL' : {'runs': [15,16], 'cond':['match','relation']},
    'SOCIAL'     : {'runs': [17,18], 'cond':['mental','rnd']}
}

# You may want to limit the subjects used during code development. This will use all subjects.

In [ ]:
# Function to load one .npy timeseries file

def load_single_timeseries(subject, experiment, run, remove_mean=True):
    """
    Load timeseries data for a single subject and single run.
  
    Args:
    subject (int):      0-based subject ID to load
    experiment (str):   Name of experiment 
    run (int):          0-based run index, across all tasks
    remove_mean (bool): If True, subtract the parcel-wise mean (typically the mean BOLD signal is not of interest)

    Returns
    ts (n_parcel x n_timepoint array): Array of BOLD data values

    """
    bold_run  = EXPERIMENTS[experiment]['runs'][run]
    bold_path = f"{HCP_DIR}/subjects/{subject}/timeseries"
    bold_file = f"bold{bold_run}_Atlas_MSMAll_Glasser360Cortical.npy"
    ts = np.load(f"{bold_path}/{bold_file}")
    if remove_mean:
        ts -= ts.mean(axis=1, keepdims=True)
    return ts


def load_evs(subject, experiment, run):
    """Load EVs (explanatory variables) data for one task experiment.

    Args:
    subject (int): 0-based subject ID to load
    experiment (str) : Name of experiment

    Returns
    evs (list of lists): A list of frames associated with each condition

    """
    frames_list = []
    task_key = 'tfMRI_'+ experiment + '_'+['RL','LR'][run]
    for cond in EXPERIMENTS[experiment]['cond']:    
        ev_file  = f"{HCP_DIR}/subjects/{subject}/EVs/{task_key}/{cond}.txt"
        ev_array = np.loadtxt(ev_file, ndmin=2, unpack=True)
        ev       = dict(zip(["onset", "duration", "amplitude"], ev_array))
    
    # Determine when trial starts, rounded down
        start = np.floor(ev["onset"] / TR).astype(int)
    
    # Use trial duration to determine how many frames to include for trial
        duration = np.ceil(ev["duration"] / TR).astype(int)
    
    # Take the range of frames that correspond to this specific trial
        frames = [s + np.arange(0, d) for s, d in zip(start, duration)]
        frames_list.append(frames)
    
    return frames_list

In [ ]:
my_exp  = 'WM'
my_subj = 2
my_run  = 0

data_example = load_single_timeseries(subject = my_subj, experiment = my_exp, run = my_run, remove_mean = True)

print(data_example.shape) # 360 ROIs, 405 timepoints
print(data_example) # np.ndarray

In [ ]:
# Info about regions
regions = np.load(f"{HCP_DIR}/regions.npy").T
region_info = dict(
    name=regions[0].tolist(),
    network=regions[1],
    hemi=['Right']*int(N_PARCELS/2) + ['Left']*int(N_PARCELS/2),
)

In [ ]:
# We want ROIs in columns
regions_T = np.transpose(regions)

In [ ]:
# Check how many networks are there
from collections import Counter

# Function to get unique values
def unique(list1):
   
    # Print directly by using * symbol
    print(*Counter(list1))

unique(regions[1]) #12 networks

In [ ]:
# Load explanatory variables - No clue what they current mean
evs = load_evs(subject = my_subj, experiment = my_exp, run = my_run)

In [ ]:
# Turn regions into a pd.dataframe
df_regions = pd.DataFrame(regions_T, columns = ['ROI', 'Network', 'Hemi'])
print(df_regions)

In [ ]:
# See how many regions you have for each network
df_regions['Network'].value_counts() # Not even close to an equal number

In [ ]:
# Again, we want ROIs in columns
data_example_T = np.transpose(data_example)
data_example_df = pd.DataFrame(data_example_T, columns = [df_regions['ROI'], df_regions['Network']])
print(data_example_df.shape) #405 time points, 360 ROIs

In [ ]:
# Transform original data_example to a pd.DataFrame
data_example_df_original = pd.DataFrame(data_example)
print(data_example_df_original)

In [ ]:
# Merge ROIs and networks
Regions_Neural = df_regions.merge(data_example_df_original, left_index=True, right_index=True)

In [ ]:
Regions_Neural.head()

In [ ]:
index_Network = np.arange(0, 360)
print(index_Network)

In [ ]:
columns_Network = Regions_Neural.columns
print(columns_Network)

In [ ]:
# Uncorrected full correlations between ROIs
ROI_CorrMatrix_Full = data_example_df.corr()
print(ROI_CorrMatrix_Full)

In [ ]:
# Heatmap - Didn't run, matrix is too big
sn.heatmap(ROI_CorrMatrix_Full, annot=True)
plt.show()

In [ ]:
# Uncorrected partial correlations between ROIs
ROI_CorrMatrix_Partial = data_example_df.pcorr()
print(ROI_CorrMatrix_Partial)

In [ ]:
# The following procedure normalizes the response within ROIs over time
scaler = preprocessing.StandardScaler().fit(data_example_df)
data_example_df_zscore = scaler.transform(data_example_df)
data_example_df_zscore = pd.DataFrame(data_example_df_zscore, columns = data_example_df.columns)

In [ ]:
# Sanity check - Mean
np.mean(data_example_df_zscore['R_4'])

In [ ]:
# Sanity check - SD
np.std(data_example_df_zscore['R_4'])

#### Note:

Data for all participants and tasks (0-back and 2-back) are downloaded, pre-processed and analysed from here.

In [ ]:
# Add modules folder to Python's search path
from os import times
import sys
from pathlib import Path
from os.path import dirname, realpath, abspath
script_dir = Path(abspath(''))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
print(module_dir)
print(script_dir)

In [ ]:
import task
db_path = '{}/data/hcp_task'.format(module_dir)
group = task.Group(db_path)

In [ ]:
# Load 0-back data - X are the features, y are the labels
X_0back, y_0back = group.extract_cons(['0bk_faces', '0bk_tools', '0bk_places', '0bk_body'])

In [ ]:
# Sanity check
print(X_0back.shape, y_0back.shape)

In [ ]:
# Load 2-back data
X_2back, y_2back = group.extract_cons(['2bk_faces', '2bk_tools', '2bk_places', '2bk_body'])

In [ ]:
print(X_2back.shape, y_2back.shape)

# 28080 - 360 (number of ROIs) x 78 (length of the time series)
# 1356 - 339 (number of subjects) x 4 (number of conditions)

#### Note:

Ignore the section below.

In [ ]:
# Create separate arrays with ROI names and time stamps
ROIs = np.array(df_regions['ROI'])
ROIs_Full = pd.DataFrame(np.repeat(ROIs, 78), columns = ['ROI'])

Timepoints = np.array(range(1,79))
Timepoints_Full = pd.DataFrame(np.tile(Timepoints, 360), columns = ['Time'])

In [ ]:
# Merge the two and create a combined column
ROIs_t = ROIs_Full.merge(Timepoints_Full, left_index=True, right_index=True)
ROIs_t['Time'] = ROIs_t['Time'].astype(str)
ROIs_t['ROI_t'] = ROIs_t['ROI'].str.cat(ROIs_t['Time'], sep ='_')

In [ ]:
# Sanity check
print(ROIs_t)

In [ ]:
# Convert the features space to pd.DataFrame
X_0back_df = pd.DataFrame(X_0back, columns = [ROIs_t['ROI_t']])
print(X_0back_df)

In [ ]:
# Add the following column: Participant 
X_0back_df['Participant'] = np.repeat(np.array(range(0,339)), 4)

In [ ]:
# Add the following column: Condition
X_0back_df['Condition'] = pd.Series(np.tile(np.array(['faces', 'tools', 'places', 'body']), 339))

In [ ]:
# Sanity check
print(X_0back_df.head())
X_0back_df.dtypes

In [ ]:
# Test if you can run classifiers in this data structure
from sklearn.linear_model import LogisticRegressionCV

# GLM
Classification = LogisticRegressionCV(cv = 8, random_state = 0, multi_class = 'multinomial', max_iter = 10000).fit(X_0back_df.iloc[:, 0:28], X_0back_df.iloc[:, 28081])

In [ ]:
# SVM
from sklearn import svm

svm.SVC().fit(X_0back_df.iloc[:, 0:28], X_0back_df.iloc[:, 28081])

#### Note:

Functional code for reshaping the dataframe can be found below.

In [ ]:
rois = regions[0]
subjects = np.arange(339)
conditions_0back = ['faces', 'tools', 'places', 'body']
conditions_2back = ['faces', 'tools', 'places', 'body']

In [ ]:
# Reshape the dataframe
index_0back = []
for cond in conditions_0back:
    for subj in subjects:
        for roi in rois:
            index_0back.append((cond, subj, roi))
len(index_0back)

In [ ]:
index_2back = []
for cond in conditions_2back:
    for subj in subjects:
        for roi in rois:
            index_2back.append((cond, subj, roi))
len(index_2back)

In [ ]:
# Reshape the 0-back task dataframe
X_new_0back = X_0back.reshape(488160, 78)

index_0back = pd.MultiIndex.from_tuples(index_0back)
X_df_0back = pd.DataFrame(X_new_0back, index = index_0back)
X_df_0back.index.names = ['Condition', 'Subject_id', 'ROI']

In [ ]:
# Sanity check
print(X_df_0back.head())
print(X_df_0back.shape)

In [ ]:
# Reshape the 2-back task dataframe
X_new_2back = X_2back.reshape(488160, 78)

index_2back = pd.MultiIndex.from_tuples(index_2back)
X_df_2back = pd.DataFrame(X_new_2back, index = index_2back)
X_df_2back.index.names = ['Condition', 'Subject_id', 'ROI']

In [ ]:
# Sanity check
print(X_df_2back.head())
print(X_df_2back.shape)

In [ ]:
# Run to unpack indices (to convert them to columns) - 0-back
X_df_0back.reset_index(level=0, inplace=True)
X_df_0back.reset_index(level=0, inplace=True)
X_df_0back.reset_index(level=0, inplace=True)

In [ ]:
# Run to unpack indices (to convert them to columns) - 2-back
X_df_2back.reset_index(level=0, inplace=True)
X_df_2back.reset_index(level=0, inplace=True)
X_df_2back.reset_index(level=0, inplace=True)

In [ ]:
# Add the network column
df_regions.reset_index(level=0, inplace=True)
print(df_regions)

In [ ]:
# Sanity check
print(X_df_0back.head())
print(X_df_0back.shape)

In [ ]:
# Adding the information about the brain network to the data frame
X_0back_full = X_df_0back.merge(df_regions, left_on='ROI', right_on='ROI')
X_2back_full = X_df_2back.merge(df_regions, left_on='ROI', right_on='ROI')

In [ ]:
# Sanity check
print(X_0back_full.head())
print(X_2back_full.head())

In [286]:
# Group by condition, network and participant and calculate the mean network activity at each of the 78 time steps
X_0back_full_net = X_0back_full.groupby(['Subject_id', 'Condition', 'Network']).mean()
X_2back_full_net = X_2back_full.groupby(['Subject_id', 'Condition', 'Network']).mean()

In [ ]:
# 339 (subj) x 4 (cond) x 12 (net) = 16272 rows
X_0back_full_net.shape

In [288]:
# Sanity check
X_0back_full_net.head()

0             1             2  \
Subject_id Condition Network                                                  
0          body      Auditory      10570.910667  10559.482667  10558.042667   
                     Cingulo-Oper  11404.612143  11397.947500  11405.201250   
                     Default       10302.066087  10315.360435  10316.175217   
                     Dorsal-atten  10436.618571  10437.545714  10429.855714   
                     Frontopariet  10976.498000  10976.870400  10979.375600   

                                              3             4             5  \
Subject_id Condition Network                                                  
0          body      Auditory      10557.971333  10569.071333  10575.828667   
                     Cingulo-Oper  11412.923036  11407.661429  11411.467500   
                     Default       10319.676522  10305.984783  10312.207391   
                     Dorsal-atten  10442.552857  10440.542857  10467.337143   
                     Frontopariet  10989.695800  10992.549800  11001.456600   

                                              6             7             8  \
Subject_id Condition Network                                                  
0          body      Auditory      10597.588667  10548.816000  10557.684667   
                     Cingulo-Oper  11409.143214  11409.967500  11416.129107   
                     Default       10331.403043  10331.197391  10324.156087   
                     Dorsal-atten  10493.134286  10476.871429  10461.774286   
                     Frontopariet  11014.737800  10986.691000  10981.526000   

                                              9  ...            69  \
Subject_id Condition Network                     ...                 
0          body      Auditory      10530.911333  ...  10533.365333   
                     Cingulo-Oper  11426.297857  ...  11197.067857   
                     Default       10331.242609  ...  10348.643913   
                     Dorsal-atten  10481.645714  ...  10351.944286   
                     Frontopariet  10992.998000  ...  10639.027400   

                                             70            71            72  \
Subject_id Condition Network                                                  
0          body      Auditory      10545.706667  10537.018667  10525.060667   
                     Cingulo-Oper  11194.273929  11182.231071  11174.226964   
                     Default       10341.022174  10325.472609  10336.795217   
                     Dorsal-atten  10383.840000  10349.467143  10362.028571   
                     Frontopariet  10635.935400  10619.900600  10626.532600   

                                             73            74            75  \
Subject_id Condition Network                                                  
0          body      Auditory      10566.845333  10558.225333  10555.538667   
                     Cingulo-Oper  11193.356786  11200.759107  11195.005714   
                     Default       10330.170000  10345.498696  10332.387391   
                     Dorsal-atten  10356.758571  10358.641429  10358.995714   
                     Frontopariet  10621.213600  10636.832000  10629.505400   

                                             76            77       index  
Subject_id Condition Network                                               
0          body      Auditory      10565.348000  10548.770667  207.266667  
                     Cingulo-Oper  11199.488214  11212.151607  186.892857  
                     Default       10347.660435  10340.260435  160.565217  
                     Dorsal-atten  10341.261429  10370.198571  193.857143  
                     Frontopariet  10625.407200  10645.620800  195.940000  

[5 rows x 79 columns]

In [289]:
# Return indices to columns (0-back)
X_0back_full_net.reset_index(level=0, inplace=True)
X_0back_full_net.reset_index(level=0, inplace=True)
X_0back_full_net.reset_index(level=0, inplace=True)

In [290]:
# Return indices to columns (2-back)
X_2back_full_net.reset_index(level=0, inplace=True)
X_2back_full_net.reset_index(level=0, inplace=True)
X_2back_full_net.reset_index(level=0, inplace=True)

In [ ]:
# Sanity check
X_0back_full_net.head()

In [291]:
# Sanity check
X_2back_full_net.head()

,Network,Condition,Subject_id,0,1,2,3,4,5,6,...,69,70,71,72,73,74,75,76,77,index
0,Auditory,body,0,10513.302667,10510.148000,10535.079333,10521.920000,10515.240667,10529.814667,10548.097333,...,10548.404667,10581.230667,10578.819333,10552.318000,10553.810000,10557.700000,10547.729333,10555.196000,10572.920667,207.266667
1,Cingulo-Oper,body,0,11359.838393,11350.669643,11362.828036,11357.317143,11354.753036,11354.653036,11372.082679,...,11190.940714,11186.418750,11194.230893,11187.954643,11196.669643,11194.907500,11183.193929,11209.787500,11206.237500,186.892857
2,Default,body,0,10280.175217,10277.216522,10268.863913,10280.523478,10267.522174,10282.300870,10301.117391,...,10356.838261,10364.836957,10358.145217,10355.730000,10362.967391,10361.825217,10340.093913,10340.578696,10358.893913,160.565217
3,Dorsal-atten,body,0,10407.442857,10426.668571,10380.267143,10386.244286,10394.920000,10415.598571,10426.321429,...,10384.895714,10395.381429,10389.944286,10396.580000,10404.450000,10411.432857,10394.377143,10414.470000,10411.192857,193.857143
4,Frontopariet,body,0,10888.065600,10873.332200,10855.890200,10869.319600,10884.365800,10897.929400,10929.934000,...,10687.228600,10686.270600,10684.482400,10675.743600,10698.277000,10689.713600,10680.966000,10680.650800,10679.955400,195.940000


In [292]:
# Delete the column you don't need
del X_0back_full_net["index"]
del X_2back_full_net["index"]

#### Note:

Ignore the code below - DO NOT rename the columns as that will throw an error when you try to subtract the 2-back and 0-back dataframes.

In [ ]:
# Rename time point columns
cols_0back = X_0back_full_net.columns
cols_2back = X_2back_full_net.columns

In [ ]:
cols_0back = np.asarray(cols_0back, dtype=str)
cols_2back = np.asarray(cols_2back, dtype=str)

In [ ]:
cols_0back = np.core.defchararray.add(cols_0back, '_0_back')
cols_2back = np.core.defchararray.add(cols_2back, '_2_back')

cols_0back[0] = 'Network'
cols_0back[1] = 'Condition'
cols_0back[2] = 'Subject_id'

cols_2back[0] = 'Network'
cols_2back[1] = 'Condition'
cols_2back[2] = 'Subject_id'

In [ ]:
# Sanity check
print(cols_0back)
print(cols_2back)

In [ ]:
# Change the column names
X_0back_full_net.columns = cols_0back
X_2back_full_net.columns = cols_2back

In [ ]:
# Sanity check
X_0back_full_net.head()

In [ ]:
# Sanity check
X_2back_full_net.head()

In [ ]:
# Merge 0-back and 2-back tasks
X_0back_2back = X_0back_full_net.merge(X_2back_full_net, left_on=['Network', 'Condition', 'Subject_id'], right_on=['Network', 'Condition', 'Subject_id'])

In [279]:
# Sanity check
X_0back_2back.head(12)

,Network,Condition,Subject_id,0_0_back,1_0_back,2_0_back,3_0_back,4_0_back,5_0_back,6_0_back,...,68_2_back,69_2_back,70_2_back,71_2_back,72_2_back,73_2_back,74_2_back,75_2_back,76_2_back,77_2_back
0,Auditory,body,0,10570.910667,10559.482667,10558.042667,10557.971333,10569.071333,10575.828667,10597.588667,...,10547.381333,10548.404667,10581.230667,10578.819333,10552.318000,10553.810000,10557.700000,10547.729333,10555.196000,10572.920667
1,Cingulo-Oper,body,0,11404.612143,11397.947500,11405.201250,11412.923036,11407.661429,11411.467500,11409.143214,...,11184.943571,11190.940714,11186.418750,11194.230893,11187.954643,11196.669643,11194.907500,11183.193929,11209.787500,11206.237500
2,Default,body,0,10302.066087,10315.360435,10316.175217,10319.676522,10305.984783,10312.207391,10331.403043,...,10348.343478,10356.838261,10364.836957,10358.145217,10355.730000,10362.967391,10361.825217,10340.093913,10340.578696,10358.893913
3,Dorsal-atten,body,0,10436.618571,10437.545714,10429.855714,10442.552857,10440.542857,10467.337143,10493.134286,...,10369.707143,10384.895714,10395.381429,10389.944286,10396.580000,10404.450000,10411.432857,10394.377143,10414.470000,10411.192857
4,Frontopariet,body,0,10976.498000,10976.870400,10979.375600,10989.695800,10992.549800,11001.456600,11014.737800,...,10657.163000,10687.228600,10686.270600,10684.482400,10675.743600,10698.277000,10689.713600,10680.966000,10680.650800,10679.955400
5,Language,body,0,9925.780435,9913.356957,9924.505217,9939.133913,9939.866957,9955.767826,9963.792174,...,9893.864348,9920.772174,9928.915652,9931.318696,9928.613478,9937.010870,9935.303043,9926.193043,9937.593478,9927.494348
6,Orbito-Affec,body,0,11250.911667,11217.478333,11135.265000,11192.260000,11271.091667,11274.868333,11228.146667,...,11081.855000,11162.095000,11090.776667,11141.870000,11076.471667,11084.970000,11133.713333,11065.673333,11107.895000,11060.241667
7,Posterior-Mu,body,0,11451.495325,11451.021688,11464.089351,11454.116494,11458.152078,11473.140519,11491.734805,...,11264.012208,11289.629221,11302.934156,11291.496883,11307.695584,11311.298052,11302.237792,11265.789091,11280.992597,11294.767922
8,Somatomotor,body,0,9633.551795,9621.335641,9624.020000,9642.984615,9636.876154,9635.671026,9634.871026,...,9501.709487,9510.855641,9502.592051,9517.827949,9514.188718,9510.324872,9509.293077,9505.212308,9511.134872,9513.292051
9,Ventral-Mult,body,0,7792.907500,7808.045000,7822.752500,7794.747500,7806.350000,7844.330000,7816.722500,...,8166.832500,8227.540000,8215.435000,8201.690000,8212.675000,8242.832500,8212.300000,8204.787500,8196.377500,8206.645000


#### Note:

Create a contrast dataframe.

In [294]:
# Contrast 2-back - 0-back
X_0back_full_net = X_0back_full_net.set_index(['Network', 'Condition', 'Subject_id'])
X_2back_full_net = X_2back_full_net.set_index(['Network', 'Condition', 'Subject_id'])

In [295]:
# Sanity check
X_0back_full_net.head()

,,,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
Network,Condition,Subject_id,,,,,,,,,,,,,,,,,,,,,
Auditory,body,0,10570.910667,10559.482667,10558.042667,10557.971333,10569.071333,10575.828667,10597.588667,10548.816000,10557.684667,10530.911333,...,10537.397333,10533.365333,10545.706667,10537.018667,10525.060667,10566.845333,10558.225333,10555.538667,10565.348000,10548.770667
Cingulo-Oper,body,0,11404.612143,11397.947500,11405.201250,11412.923036,11407.661429,11411.467500,11409.143214,11409.967500,11416.129107,11426.297857,...,11191.768214,11197.067857,11194.273929,11182.231071,11174.226964,11193.356786,11200.759107,11195.005714,11199.488214,11212.151607
Default,body,0,10302.066087,10315.360435,10316.175217,10319.676522,10305.984783,10312.207391,10331.403043,10331.197391,10324.156087,10331.242609,...,10332.352174,10348.643913,10341.022174,10325.472609,10336.795217,10330.170000,10345.498696,10332.387391,10347.660435,10340.260435
Dorsal-atten,body,0,10436.618571,10437.545714,10429.855714,10442.552857,10440.542857,10467.337143,10493.134286,10476.871429,10461.774286,10481.645714,...,10344.114286,10351.944286,10383.840000,10349.467143,10362.028571,10356.758571,10358.641429,10358.995714,10341.261429,10370.198571
Frontopariet,body,0,10976.498000,10976.870400,10979.375600,10989.695800,10992.549800,11001.456600,11014.737800,10986.691000,10981.526000,10992.998000,...,10622.288000,10639.027400,10635.935400,10619.900600,10626.532600,10621.213600,10636.832000,10629.505400,10625.407200,10645.620800


In [296]:
# Sanity check
X_2back_full_net.head()

,,,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
Network,Condition,Subject_id,,,,,,,,,,,,,,,,,,,,,
Auditory,body,0,10513.302667,10510.148000,10535.079333,10521.920000,10515.240667,10529.814667,10548.097333,10561.133333,10554.373333,10564.174000,...,10547.381333,10548.404667,10581.230667,10578.819333,10552.318000,10553.810000,10557.700000,10547.729333,10555.196000,10572.920667
Cingulo-Oper,body,0,11359.838393,11350.669643,11362.828036,11357.317143,11354.753036,11354.653036,11372.082679,11388.209643,11399.702857,11410.989464,...,11184.943571,11190.940714,11186.418750,11194.230893,11187.954643,11196.669643,11194.907500,11183.193929,11209.787500,11206.237500
Default,body,0,10280.175217,10277.216522,10268.863913,10280.523478,10267.522174,10282.300870,10301.117391,10311.949130,10314.795652,10310.440435,...,10348.343478,10356.838261,10364.836957,10358.145217,10355.730000,10362.967391,10361.825217,10340.093913,10340.578696,10358.893913
Dorsal-atten,body,0,10407.442857,10426.668571,10380.267143,10386.244286,10394.920000,10415.598571,10426.321429,10443.237143,10453.041429,10457.281429,...,10369.707143,10384.895714,10395.381429,10389.944286,10396.580000,10404.450000,10411.432857,10394.377143,10414.470000,10411.192857
Frontopariet,body,0,10888.065600,10873.332200,10855.890200,10869.319600,10884.365800,10897.929400,10929.934000,10956.846200,10962.209800,10965.212200,...,10657.163000,10687.228600,10686.270600,10684.482400,10675.743600,10698.277000,10689.713600,10680.966000,10680.650800,10679.955400


In [297]:
# Create a contrast dataframe
X_2back_0back_contr = X_2back_full_net - X_0back_full_net

In [301]:
X_2back_0back_contr.head()

,Subject_id,Condition,Network,0,1,2,3,4,5,6,...,68,69,70,71,72,73,74,75,76,77
0,0,body,Auditory,-57.608000,-49.334667,-22.963333,-36.051333,-53.830667,-46.014000,-49.491333,...,9.984000,15.039333,35.524000,41.800667,27.257333,-13.035333,-0.525333,-7.809333,-10.152000,24.150000
1,0,body,Cingulo-Oper,-44.773750,-47.277857,-42.373214,-55.605893,-52.908393,-56.814464,-37.060536,...,-6.824643,-6.127143,-7.855179,11.999821,13.727679,3.312857,-5.851607,-11.811786,10.299286,-5.914107
2,0,body,Default,-21.890870,-38.143913,-47.311304,-39.153043,-38.462609,-29.906522,-30.285652,...,15.991304,8.194348,23.814783,32.672609,18.934783,32.797391,16.326522,7.706522,-7.081739,18.633478
3,0,body,Dorsal-atten,-29.175714,-10.877143,-49.588571,-56.308571,-45.622857,-51.738571,-66.812857,...,25.592857,32.951429,11.541429,40.477143,34.551429,47.691429,52.791429,35.381429,73.208571,40.994286
4,0,body,Frontopariet,-88.432400,-103.538200,-123.485400,-120.376200,-108.184000,-103.527200,-84.803800,...,34.875000,48.201200,50.335200,64.581800,49.211000,77.063400,52.881600,51.460600,55.243600,34.334600


In [300]:
# Return indices to columns (2-back)
X_2back_0back_contr.reset_index(level=0, inplace=True)
X_2back_0back_contr.reset_index(level=0, inplace=True)
X_2back_0back_contr.reset_index(level=0, inplace=True)

In [ ]:
# TO DO:
# NORMALIZING DATA - It is very important to figure out what is meant by this and how to execute it.
# NORMALIZING DATA - This might probably need to be done before creating contrasts or anything.
# WITHIN-NETWORK CLASSIFICATION - Doing the classification within each network + Regularization?
# CROSS-VALIDATION - Are we 'manually' splitting the data into train/test or is the function doing that for us?
# REGULARIZATION

In [308]:
# sklearn imports
from sklearn.model_selection import train_test_split

In [ ]:
# GLM - THIS IS NOT SPECIFIED CORRECTLY, IT NEEDS TO BE UPDATED.
Classification_Contrast = LogisticRegressionCV(cv = 8, random_state = 0, multi_class = 'multinomial', max_iter = 10000).fit(X_2back_0back_contr.iloc[:, 3:81], X_2back_0back_contr.iloc[:, 1])
